# QoL Monitor - Modeling

For the Machine Learning algorithms modeling, we decided to use the Scikit-learn toolbox due to its high acceptance in the scientific community and the consistency of its results.

## Required Imports

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## Getting Processed Data

In [2]:
base_path = '/kaggle/input/self-reported-qol/20230625-processed-'

df_physical      = pd.read_csv(base_path + 'physical-qol.csv')
df_psychological = pd.read_csv(base_path + 'psychological-qol.csv')

In [3]:
df_physical.drop('id', axis=1, inplace=True)         # id
df_physical.drop('day', axis=1, inplace=True)        # day

df_psychological.drop('id', axis=1, inplace=True)    # id
df_psychological.drop('day', axis=1, inplace=True)   # day

In [4]:
phy_all_tmp = df_physical.copy()
phy_init_set_tmp = phy_all_tmp.query("group in ('Initial Set')").copy()
phy_ufpi_ufc_tmp = phy_all_tmp.query("group in ('UFPI', 'UFC')").copy()

psy_all_tmp = df_psychological.copy()
psy_init_set_tmp = psy_all_tmp.query("group in ('Initial Set')").copy()
psy_ufpi_ufc_tmp = psy_all_tmp.query("group in ('UFPI', 'UFC')").copy()

In [5]:
phy_all_tmp.drop("group", axis=1, inplace=True)
phy_init_set_tmp.drop("group", axis=1, inplace=True)
phy_ufpi_ufc_tmp.drop("group", axis=1, inplace=True)

psy_all_tmp.drop("group", axis=1, inplace=True)
psy_init_set_tmp.drop("group", axis=1, inplace=True)
psy_ufpi_ufc_tmp.drop("group", axis=1, inplace=True)

In [6]:
# Separating predictors from the values to be predicted.
phy_all           = phy_all_tmp.drop("phy_ref_score", axis=1)
phy_all_pred      = phy_all_tmp["phy_ref_score"].copy()
phy_init_set      = phy_init_set_tmp.drop("phy_ref_score", axis=1)
phy_init_set_pred = phy_init_set_tmp["phy_ref_score"].copy()
phy_ufpi_ufc      = phy_ufpi_ufc_tmp.drop("phy_ref_score", axis=1)
phy_ufpi_ufc_pred = phy_ufpi_ufc_tmp["phy_ref_score"].copy()

psy_all           = psy_all_tmp.drop("psy_ref_score", axis=1)
psy_all_pred      = psy_all_tmp["psy_ref_score"].copy()
psy_init_set      = psy_init_set_tmp.drop("psy_ref_score", axis=1)
psy_init_set_pred = psy_init_set_tmp["psy_ref_score"].copy()
psy_ufpi_ufc      = psy_ufpi_ufc_tmp.drop("psy_ref_score", axis=1)
psy_ufpi_ufc_pred = psy_ufpi_ufc_tmp["psy_ref_score"].copy()

In [7]:
features_selected = ['height', 'weight', 'income', 'children', 'specificage', 'familyarr', 'maritalstatus_single',
                     'profession_parttimeworker', 'profession_selfemployed', 'profession_student',                      
                     'sdnn', 'max_hr', 'nni_20', 'mean_hr', 'cvsd', 'pnni_20',
                     'walking', 'invehicle', 'calories', 'steps',
                     'health', 'social', 'communication', 'business', 'finance', 'video', 'photography', 
                     'productivity', 'education', 'games', 'music', 'entertainment', 'books']

## Training regression models

In [8]:
cols_result_df = ['exec_index', 'name', 'model', 'n_folds', 'feature_selection', 'mae_mean', 'mae_std', 'rmse_mean', 'rmse_std', 'r2_mean', 'r2_std', 'time(s)']

In [9]:
def getDataset(domain_group):
    return {
        'phy_all': (phy_all, phy_all_pred),
        'psy_all': (psy_all, psy_all_pred),
        
        'phy_init_set': (phy_init_set, phy_init_set_pred),
        'psy_init_set': (psy_init_set, psy_init_set_pred),
        
        'phy_ufpi_ufc': (phy_ufpi_ufc, phy_ufpi_ufc_pred),
        'psy_ufpi_ufc': (psy_ufpi_ufc, psy_ufpi_ufc_pred),
    }[domain_group] 

In [10]:
def rmsle_cv(exec_index, name, model, n_folds, X, y, feat_sel):
    start = time.time()
    
    X_filtered = X
    if(feat_sel == 'SULOV'):
        X_filtered = X[features_selected].copy()
    
    kf = KFold(n_folds, shuffle = True, random_state = 42).get_n_splits(X.values)
    rmse = np.sqrt(-cross_val_score(model, X_filtered.values, y, scoring = "neg_mean_squared_error", cv = kf))
    mae  = -cross_val_score(model, X_filtered.values, y, scoring = "neg_mean_absolute_error", cv = kf)
    r2 = cross_val_score(model, X_filtered.values, y, scoring = "r2", cv = kf)
    
    end = time.time()
    exec_time = end - start
    scores = pd.Series([exec_index, name, model, n_folds, feat_sel,
                      round(mae.mean(), 4),  round(mae.std(), 4), 
                      round(rmse.mean(), 4), round(rmse.std(), 4), 
                      round(r2.mean(), 4),   round(r2.std(), 4), 
                      round(exec_time, 4)],  index=cols_result_df)
    print('MAE:', round(mae.mean(), 4), 'RMSE:',  round(rmse.mean(), 4), 'R^2:', round(r2.mean(), 4), '\n')
    return scores

In [11]:
def train_model(name, model, n_folds, X, y, executions, feat_sel):
    df = pd.DataFrame(columns = cols_result_df)
    for exec_index in range(executions):
        print('|_ Training', name, 'Execution', exec_index, '...')
        result = rmsle_cv(exec_index, name, model, n_folds, X, y, feat_sel)
        df = pd.concat([df, result.to_frame().T], ignore_index=True)
    return df

In [12]:
kfold = 10
executions = 30
feat_to_select = 20

def modelingRegressors(domain_group):
    print('Modeling regressors for', domain_group)
    
    dfs = []
    X, y = getDataset(domain_group)
    alg_name = 'GBoost'
    algorithm = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state=5)
    
    skb = SelectKBest(k = feat_to_select)
    pca = PCA(n_components = feat_to_select)
    
    simple_pipe  = Pipeline([('regressor', algorithm)])
    complex_pipe = Pipeline([('transformer', StandardScaler()), ('feat_selection', skb), ('regressor', algorithm)])
    
    dfs.append(train_model(alg_name, simple_pipe,  kfold, X, y, executions, 'None'))
    dfs.append(train_model(alg_name, simple_pipe,  kfold, X, y, executions, 'SULOV'))
    dfs.append(train_model(alg_name, complex_pipe, kfold, X, y, executions, 'None'))      
        
    return pd.concat(dfs)

In [13]:
#result = modelingRegressors('phy_all')
#result.to_csv('20230703-modeling-results-gboost-regression-physical.csv', index = False)

In [14]:
result = modelingRegressors('psy_all')
result.to_csv('20230703-modeling-results-gboost-regression-psychological.csv', index = False)

Modeling regressors for psy_all
|_ Training GBoost Execution 0 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 1 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 2 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 3 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 4 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 5 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 6 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 7 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 8 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 9 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 10 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 11 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoost Execution 12 ...
MAE: 5.7768 RMSE: 8.0693 R^2: 0.722 

|_ Training GBoo

/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [70 72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/featu

MAE: 6.2432 RMSE: 8.7565 R^2: 0.6689 

|_ Training GBoost Execution 1 ...


/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [70 72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/featu

MAE: 6.2432 RMSE: 8.7565 R^2: 0.6689 

|_ Training GBoost Execution 2 ...


...

MAE: 6.2432 RMSE: 8.7565 R^2: 0.6689 

|_ Training GBoost Execution 28 ...


/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [70 72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/featu

MAE: 6.2432 RMSE: 8.7565 R^2: 0.6689 

|_ Training GBoost Execution 29 ...


/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [70 72 78 79] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/opt/conda/lib/python3.10/site-packages/sklearn/featu

MAE: 6.2432 RMSE: 8.7565 R^2: 0.6689 

